# Import
## Add src to import search path

In [1]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent
SRC = Path.joinpath(ROOT, "src")

sys.path.append(str(SRC))

## Import necessary libraries

In [2]:
from dataset import Preprocessor
from vae import VAE
from evaluation import Evaluation

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import tensorflow as tf
import numpy as np

# Preprocess data

In [3]:
preprocessor = Preprocessor()
x, y = preprocessor.compute_melSpectrogram()
print(x.shape, y.shape)

Processing Dataset: 1000it [00:23, 42.66it/s]


(1000, 128, 1292) (1000,)


# Models

## Enable GPU memory growth

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled")
    except RuntimeError as e:
        print("Memory growth failed:", e)

Memory growth enabled


## Declare variables

In [5]:
EPOCH = 10
BATCH_SIZE = 32
LATENT_DIM = 16
OPTIMIZER_VAE = tf.keras.optimizers.Adam(1e-4)
OPTIMIZER_CVAE = tf.keras.optimizers.Adam(1e-3)

## PCA

### Prepare data

In [6]:
x_flat = x.reshape((x.shape[0], -1))
input_dim = x_flat.shape[1]
train_X = tf.data.Dataset.from_tensor_slices(x_flat.astype('float32')).shuffle(x.shape[0]).batch(BATCH_SIZE)

### PCA training

In [7]:
pca = PCA(n_components=0.9, random_state=42)
x_pca = pca.fit_transform(x_flat)

### Kmeans training

In [8]:
kmeans_pca = KMeans(n_clusters=10, random_state=42, n_init=10)
kmeans_pca.fit(x_pca)

KMeans(n_clusters=10, n_init=10, random_state=42)

### Evaluation

In [9]:
# Evaluate
clusters_pca = kmeans_pca.predict(x_pca)

evaluation_pca = Evaluation()
evaluation_pca.tSNE(x_pca, clusters_pca, "../results/pca_tsne.pdf")
sh_pca = evaluation_pca.silhouette(x_pca, clusters_pca)
ch_pca = evaluation_pca.calinski_harabasz(x_pca, clusters_pca)

# Write results
with open("../results/pca_score.txt", 'w') as file:
    file.write(f'SH score PCA: {str(sh_pca)}\nCH score PCA: {str(ch_pca)}')

## Vanilla VAE

### Prepare data

In [10]:
x_flat = x.reshape((x.shape[0], -1))
input_dim = x_flat.shape[1]
train_X = tf.data.Dataset.from_tensor_slices(x_flat.astype('float32')).shuffle(x.shape[0]).batch(BATCH_SIZE)

### VAE training

In [11]:
vae = VAE(input_dim, LATENT_DIM, "VAE")
vae.fit(train_dataset=train_X, epochs=EPOCH, optimizer=OPTIMIZER_VAE)

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch: 1, Training ELBO: -206836.6875
Epoch: 2, Training ELBO: -46777.01171875
Epoch: 3, Training ELBO: -80275.328125
Epoch: 4, Training ELBO: -63043.23828125
Epoch: 5, Training ELBO: -54130.375
Epoch: 6, Training ELBO: -93437.1171875
Epoch: 7, Training ELBO: -46682.83203125
Epoch: 8, Training ELBO: -34731.89453125
Epoch: 9, Training ELBO: -36078.7578125
Epoch: 10, Training ELBO: -35698.09375


### Kmeans training

In [12]:
mean, logvar = vae.encode(x_flat.astype('float32'))
vae_latent_features = mean.numpy()

kmeans_vae = KMeans(n_clusters=len(np.unique(y)), random_state=42)
kmeans_vae.fit(vae_latent_features)

KMeans(n_clusters=10, random_state=42)

### Evaluation

In [13]:
# Evaluate
clusters_vae = kmeans_vae.predict(vae_latent_features)

evaluation_vae = Evaluation()
evaluation_vae.tSNE(vae_latent_features, clusters_vae, "../results/vae_tsne.pdf")
sh_vae = evaluation_vae.silhouette(vae_latent_features, clusters_vae)
ch_vae = evaluation_vae.calinski_harabasz(vae_latent_features, clusters_vae)

# Write results
with open("../results/vae_score.txt", 'w') as file:
    file.write(f'SH score VAE: {str(sh_vae)}\nCH score VAE: {str(ch_vae)}')

## Convolutional VAE

### Prepare data

In [6]:
x_convo = x.reshape((x.shape[0], x.shape[1], x.shape[2], 1))
input_dim = (x_convo.shape[1], x_convo.shape[2], 1)
train_X = tf.data.Dataset.from_tensor_slices(x_convo.astype('float32')).shuffle(x_convo.shape[0]).batch(BATCH_SIZE)

### VAE training

In [7]:
cvae = VAE(input_dim, LATENT_DIM, "CVAE")
cvae.fit(train_dataset=train_X, epochs=EPOCH, optimizer=OPTIMIZER_CVAE)

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch: 1, Training ELBO: -780986.3125
Epoch: 2, Training ELBO: -33096.21484375
Epoch: 3, Training ELBO: -1274.7606201171875
Epoch: 4, Training ELBO: -351.458251953125
Epoch: 5, Training ELBO: -238.83926391601562
Epoch: 6, Training ELBO: -189.35324096679688
Epoch: 7, Training ELBO: -173.8084259033203
Epoch: 8, Training ELBO: -163.55686950683594
Epoch: 9, Training ELBO: -157.33546447753906
Epoch: 10, Training ELBO: -154.00474548339844


### Kmeans training

In [8]:
mean, logvar = cvae.encode(x_convo.astype('float32'))
z = cvae.reparameterize(mean, logvar)
cvae_latent_features = z.numpy()

kmeans_cvae = KMeans(n_clusters=len(np.unique(y)), random_state=42)
kmeans_cvae.fit(cvae_latent_features)

KMeans(n_clusters=10, random_state=42)

### Evaluation

In [9]:
# Evaluate
clusters_cvae = kmeans_cvae.predict(cvae_latent_features)

evaluation_cvae = Evaluation()
evaluation_cvae.tSNE(cvae_latent_features, clusters_cvae, "../results/cvae_tsne.pdf")
sh_cvae = evaluation_cvae.silhouette(cvae_latent_features, clusters_cvae)
ch_cvae = evaluation_cvae.calinski_harabasz(cvae_latent_features, clusters_cvae)

# Write results
with open("../results/cvae_score.txt", 'w') as file:
    file.write(f'SH score CVAE: {str(sh_cvae)}\nCH score CVAE: {str(ch_cvae)}')

## Beta VAE

### Prepare data

In [10]:
x_convo = x.reshape((x.shape[0], x.shape[1], x.shape[2], 1))
input_dim = (x_convo.shape[1], x_convo.shape[2], 1)
train_X = tf.data.Dataset.from_tensor_slices(x_convo.astype('float32')).shuffle(x_convo.shape[0]).batch(BATCH_SIZE)

### VAE train

In [11]:
bvae = VAE(input_dim, LATENT_DIM, "CVAE", use_beta=True)
bvae.fit(train_dataset=train_X, epochs=EPOCH, optimizer=OPTIMIZER_CVAE)

Epoch: 1, Training ELBO: -3503128.5
Epoch: 2, Training ELBO: -607.4169921875
Epoch: 3, Training ELBO: -207.60960388183594
Epoch: 4, Training ELBO: -178.7772979736328
Epoch: 5, Training ELBO: -168.03854370117188
Epoch: 6, Training ELBO: -163.20985412597656
Epoch: 7, Training ELBO: -161.31114196777344
Epoch: 8, Training ELBO: -162.6117706298828
Epoch: 9, Training ELBO: -158.45736694335938
Epoch: 10, Training ELBO: -159.47119140625


### Kmeans train

In [12]:
mean, logvar = bvae.encode(x_convo.astype('float32'))
z = bvae.reparameterize(mean, logvar)
bvae_latent_features = z.numpy()

kmeans_bvae = KMeans(n_clusters=len(np.unique(y)), random_state=42)
kmeans_bvae.fit(bvae_latent_features)

KMeans(n_clusters=10, random_state=42)

### Evaluation

In [13]:
# Evaluate
clusters_bvae = kmeans_bvae.predict(bvae_latent_features)

evaluation_bvae = Evaluation()
evaluation_bvae.tSNE(bvae_latent_features, clusters_bvae, "../results/bvae_tsne.pdf")
sh_bvae = evaluation_bvae.silhouette(bvae_latent_features, clusters_bvae)
ch_bvae = evaluation_bvae.calinski_harabasz(bvae_latent_features, clusters_bvae)

# Write results
with open("../results/bvae_score.txt", 'w') as file:
    file.write(f'SH score BVAE: {str(sh_bvae)}\nCH score BVAE: {str(ch_bvae)}')